# **DATA WRANGLING**
This file attemps to gather, collect, and transform all the raw datasets from the source attached below in order to analyse the data avilable and proceed with it. 
The following processes will be dealt with: 
1. Reading the data and transforming variables
2. Dealing with missing values
3. Data exploration
4. Reshaping data
5. Filtering data

Source: https://datos.madrid.es/sites/v/index.jsp?vgnextoid=374512b9ace9f310VgnVCM100000171f5a0aRCRD&buscar=true&Texto=BiciMAD&Sector=&Formato=&Periodicidad=&orderByCombo=CONTENT_INSTANCE_NAME_DECODE

#### **LIBRARIES**

In [2]:
import pandas as pd 

#### **READ DATA**

Situacion Estaciones BiciMAD

In [3]:
sit_201909 = pd.read_json('../Data/Situacion Estaciones/201907.json')
sit_201909.head()

,_id,stations
0,2019-07-01T00:29:26.018083,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
1,2019-07-01T03:29:30.886274,"[{'activate': 1, 'name': 'Madrid Sol 1A', 'res..."
2,2019-07-01T04:29:32.858592,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
3,2019-07-01T05:29:34.913030,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
4,2019-07-01T06:29:35.163470,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."


In [14]:
import json
# load data using Python JSON module
with open('../Data/Situacion Estaciones/201907.json','r') as f:
    data = json.loads(f.read())
df_nested_list = pd.json_normalize(data, record_path =['stations'], meta=['_id'])

In [16]:
from os import walk
path_situacionEst="../Data/Situacion Estaciones/"
f_situacionEstaciones = []
for (dirpath, dirnames, filenames) in walk(path_situacionEst):
    f_situacionEstaciones.extend(filenames)
    break


In [24]:
k=0
output = pd.DataFrame()
for file in f_situacionEstaciones:
    print(path_situacionEst+file) 
    with open(path_situacionEst+file,'r') as f:
        data = json.loads(f.read())
    df_nested_list = pd.json_normalize(data, record_path =['stations'], meta=['_id'])
    
    df_nested_list['fecha'] =df_nested_list['_id'].apply(lambda x: x[0:10])
    df_nested_list['hora'] =df_nested_list['_id'].apply(lambda x: x[11:])
    print(df_nested_list.shape)
    if k==0:
        df=df_nested_list
    else: 
        df=pd.concat([df,df_nested_list], axis=0)
        print(df.shape)
    k+=1
    
          
df.to_parquet('../Data/Situacion Estaciones/Estaciones.parquet')

../Data/Situacion Estaciones/201907.json
(135058, 16)
../Data/Situacion Estaciones/201908.json
(137274, 16)
(272332, 16)
../Data/Situacion Estaciones/201909.json
(145286, 16)
(417618, 16)
../Data/Situacion Estaciones/201910.json
(156670, 16)
(574288, 16)
../Data/Situacion Estaciones/201911.json
(151869, 16)
(726157, 16)
../Data/Situacion Estaciones/201912.json
(157620, 16)
(883777, 16)


In [25]:
estaciones=pd.read_parquet('../Data/Situacion Estaciones/Estaciones.parquet')
estaciones.shape

(883777, 16)

In [26]:
estaciones.head()

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,latitude,dock_bikes,id,_id,fecha,hora
0,1,Puerta del Sol A,0,2,24,15,1a,-3.7024255,0,Puerta del Sol nº 1,40.4168961,8,1,2019-07-01T00:29:26.018083,2019-07-01,00:29:26.018083
1,1,Puerta del Sol B,0,2,24,14,1b,-3.7024207,0,Puerta del Sol nº 1,40.4170009,9,2,2019-07-01T00:29:26.018083,2019-07-01,00:29:26.018083
2,1,Miguel Moya,0,2,24,12,2,-3.7058415,0,Calle Miguel Moya nº 1,40.4205886,10,3,2019-07-01T00:29:26.018083,2019-07-01,00:29:26.018083
3,1,Plaza Conde Suchil,0,1,18,4,3,-3.7069171,0,Plaza del Conde Suchil nº 2-4,40.4302937,11,4,2019-07-01T00:29:26.018083,2019-07-01,00:29:26.018083
4,1,Malasaña,0,0,24,17,4,-3.7025875,0,Calle Manuela Malasaña nº 5,40.4285524,3,5,2019-07-01T00:29:26.018083,2019-07-01,00:29:26.018083


In [27]:
from os import walk
path_itinerarios="../Data/Itinerarios/"
f_itinerarios = []
for (dirpath, dirnames, filenames) in walk(path_itinerarios):
    f_itinerarios.extend(filenames)
    break


Itineraios

In [28]:

itinerarios = pd.DataFrame()
for file in f_itinerarios:
    if 'parquet'in file: 
        it = pd.read_parquet(path_itinerarios+file)

    else: 
        it = pd.read_json(path_itinerarios+file)
        
    itinerarios = itinerarios.append(it, ignore_index=True)

itinerarios.to_parquet('../Data/Itinerarios/ItinerariosMerged.parquet')
        

In [37]:
itinerarios.dtypes 

_id                 object
user_day_code       object
idplug_base          int64
user_type            int64
idunplug_base        int64
travel_time          int64
idunplug_station     int64
ageRange             int64
idplug_station       int64
unplug_hourTime     object
zip_code            object
dtype: object

In [36]:
itinerarios.memory_usage()

Index                    128
_id                 14903848
user_day_code       14903848
idplug_base         14903848
user_type           14903848
idunplug_base       14903848
travel_time         14903848
idunplug_station    14903848
ageRange            14903848
idplug_station      14903848
unplug_hourTime     14903848
zip_code            14903848
dtype: int64

Bases BiciMAD

In [31]:
itinerarios.head()

,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code
0,{'$oid': '5ee862cecc8c1e839b419387'},509e6a72b7673f533912bf647d20df19305b848fbc1b89...,5,1,20,154,44,5,40,2019-07-01T00:00:00Z,28007
1,{'$oid': '5ee862cecc8c1e839b41938c'},90069a1194c3eb9e6faecaaf4eefb7209e890c0df5aeac...,8,1,17,184,57,4,129,2019-07-01T00:00:00Z,28045
2,{'$oid': '5ee862cecc8c1e839b419394'},b0ecaab6a35b7ea44d118f851f7fad5c2237e7dd51b337...,15,1,16,282,53,4,43,2019-07-01T00:00:00Z,
3,{'$oid': '5ee862cecc8c1e839b41939e'},5e1b8a8c4603ee8ac4f06aba12ff51e2c5dd105b6e5193...,18,1,18,388,56,0,13,2019-07-01T00:00:00Z,
4,{'$oid': '5ee862cecc8c1e839b4193a5'},80974c4043db61721399c43f296ce1db4436c7f107f8fc...,1,1,20,254,105,0,75,2019-07-01T00:00:00Z,


In [79]:
len(itinerarios.idplug_base.unique())

30

In [72]:
bases = pd.read_excel('../Data/bases_bicimad.xls')
bases.head()

,Número,Gis_X,Gis_Y,Fecha de Alta,Distrito,Barrio,Calle,Nº Finca,Tipo de Reserva,Número de Plazas,Longitud,Latitud,Direccion
0,001 a,440443.61,4474290.65,43803,01 CENTRO,01-06 SOL,"ALCALA, CALLE, DE",2,BiciMAD,30,-3.701998,40.417111,"ALCALA, CALLE, DE, 2"
1,001 b,440480.56,4474301.74,43867,01 CENTRO,01-06 SOL,"ALCALA, CALLE, DE",6,BiciMAD,30,-3.701564,40.417213,"ALCALA, CALLE, DE, 6"
2,2,440134.83,4474678.23,41813,01 CENTRO,01-05 UNIVERSIDAD,"MIGUEL MOYA, CALLE, DE",1,BiciMAD,24,-3.705674,40.420580,"MIGUEL MOYA, CALLE, DE, 1"
3,3,440012.98,4475760.68,41813,07 CHAMBERÍ,07-02 ARAPILES,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL",2,BiciMAD,18,-3.707212,40.430322,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL, 2"
4,4,440396.40,4475565.36,41813,01 CENTRO,01-05 UNIVERSIDAD,"MANUELA MALASAÑA, CALLE, DE",3,BiciMAD,24,-3.702674,40.428590,"MANUELA MALASAÑA, CALLE, DE, 3"


In [73]:
bases.isna().sum()

Número              0
Gis_X               0
Gis_Y               0
Fecha de Alta       0
Distrito            0
Barrio              0
Calle               0
Nº Finca            0
Tipo de Reserva     0
Número de Plazas    0
Longitud            0
Latitud             0
Direccion           0
dtype: int64

In [82]:
len(bases.Número.unique())

269